# Topolojik Özellik Çıkarımı

Bu notebook, CT görüntülerinden kalıcı homoloji kullanarak topolojik özellikler çıkarmak için gerekli kodları içerir.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import torch
import torch.nn as nn
import cv2
from scipy import ndimage
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Topolojik veri analizi kütüphaneleri
try:
    import gudhi as gd
    print("GUDHI kütüphanesi yüklendi.")
except ImportError:
    print("GUDHI kütüphanesi bulunamadı. pip install gudhi ile kurabilirsiniz.")

try:
    from gtda.homology import VietorisRipsPersistence, CubicalPersistence
    from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints
    from gtda.plotting import plot_diagram
    print("giotto-tda kütüphanesi yüklendi.")
except ImportError:
    print("giotto-tda kütüphanesi bulunamadı. pip install giotto-tda ile kurabilirsiniz.")

# Kendi modüllerimizi import et
import sys
sys.path.append('.')

plt.style.use('default')
sns.set_palette('husl')

## Kalıcı Homoloji Hesaplama Fonksiyonları

In [ ]:
class TopologicalFeatureExtractor:
    """
    CT görüntülerinden topolojik özellik çıkarımı için sınıf
    """
    
    def __init__(self, method='cubical', max_dimension=2):
        """
        Args:
            method (str): 'cubical' veya 'vietoris_rips'
            max_dimension (int): Maksimum homoloji boyutu
        """
        self.method = method
        self.max_dimension = max_dimension
        
        if method == 'cubical':
            self.persistence = CubicalPersistence(
                homology_dimensions=list(range(max_dimension + 1))
            )
        elif method == 'vietoris_rips':
            self.persistence = VietorisRipsPersistence(
                homology_dimensions=list(range(max_dimension + 1))
            )
        
        # Kalıcılık istatistik hesaplayıcıları
        self.entropy_calculator = PersistenceEntropy()
        self.amplitude_calculator = Amplitude()
        self.number_calculator = NumberOfPoints()
    
    def extract_persistence_diagram(self, image):
        """
        Görüntüden kalıcılık diagramı çıkar
        
        Args:
            image (np.array): Giriş görüntüsü
            
        Returns:
            persistence_diagram: Kalıcılık diagramı
        """
        # Görüntüyü normalize et
        if image.max() > 1.0:
            image = image / 255.0
        
        # Görüntüyü uygun boyuta getir
        if self.method == 'cubical':
            # Cubical persistence için 3D array gerekli (height, width, 1)
            if image.ndim == 2:
                image = image.reshape(1, *image.shape)
            elif image.ndim == 3 and image.shape[0] == 1:
                pass  # Zaten doğru format
            else:
                image = image.reshape(1, *image.shape[:2])
        
        # Kalıcılık diagramını hesapla
        persistence_diagram = self.persistence.fit_transform([image])
        
        return persistence_diagram[0]
    
    def extract_topological_features(self, image):
        """
        Görüntüden topolojik özellik vektörü çıkar
        
        Args:
            image (np.array): Giriş görüntüsü
            
        Returns:
            features (dict): Topolojik özellikler
        """
        # Kalıcılık diagramını al
        persistence_diagram = self.extract_persistence_diagram(image)
        
        # Özellik vektörlerini hesapla
        features = {}
        
        # Entropy
        entropy = self.entropy_calculator.fit_transform([persistence_diagram])
        features['entropy'] = entropy[0]
        
        # Amplitude
        amplitude = self.amplitude_calculator.fit_transform([persistence_diagram])
        features['amplitude'] = amplitude[0]
        
        # Number of points
        num_points = self.number_calculator.fit_transform([persistence_diagram])
        features['num_points'] = num_points[0]
        
        # Betti sayıları
        betti_numbers = self.compute_betti_numbers(persistence_diagram)
        features['betti_numbers'] = betti_numbers
        
        # Persistence landscape (basitleştirilmiş)
        landscape = self.compute_persistence_landscape(persistence_diagram)
        features['landscape'] = landscape
        
        return features, persistence_diagram
    
    def compute_betti_numbers(self, persistence_diagram, threshold=0.1):
        """
        Kalıcılık diagramından Betti sayılarını hesapla
        """
        betti_numbers = []
        
        for dim in range(self.max_dimension + 1):
            # Bu boyuttaki noktaları filtrele
            dim_points = persistence_diagram[persistence_diagram[:, 2] == dim]
            
            if len(dim_points) > 0:
                # Kalıcılığı threshold'dan büyük olanları say
                persistence = dim_points[:, 1] - dim_points[:, 0]
                significant = (persistence > threshold).sum()
                betti_numbers.append(significant)
            else:
                betti_numbers.append(0)
        
        return np.array(betti_numbers)
    
    def compute_persistence_landscape(self, persistence_diagram, resolution=100):
        """
        Basitleştirilmiş persistence landscape hesapla
        """
        if len(persistence_diagram) == 0:
            return np.zeros(resolution)
        
        # 0-boyutlu özellikler için landscape
        dim0_points = persistence_diagram[persistence_diagram[:, 2] == 0]
        
        if len(dim0_points) == 0:
            return np.zeros(resolution)
        
        # Basit landscape: her nokta için üçgen fonksiyon
        births = dim0_points[:, 0]
        deaths = dim0_points[:, 1]
        
        # Grid oluştur
        x_min, x_max = births.min(), deaths.max()
        if x_max <= x_min:
            return np.zeros(resolution)
        
        x_grid = np.linspace(x_min, x_max, resolution)
        landscape = np.zeros(resolution)
        
        for birth, death in zip(births, deaths):
            for i, x in enumerate(x_grid):
                if birth <= x <= death:
                    landscape[i] += min(x - birth, death - x)
        
        return landscape

## Görselleştirme Fonksiyonları

In [ ]:
def visualize_persistence_diagram(persistence_diagram, title="Kalıcılık Diagramı"):
    """
    Kalıcılık diagramını görselleştir
    """
    if len(persistence_diagram) == 0:
        print("Boş kalıcılık diagramı")
        return
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    
    # Farklı boyutlar için farklı renkler
    colors = ['red', 'blue', 'green']
    dimensions = ['H₀ (Bileşenler)', 'H₁ (Döngüler)', 'H₂ (Boşluklar)']
    
    for dim in range(min(3, int(persistence_diagram[:, 2].max()) + 1)):
        dim_points = persistence_diagram[persistence_diagram[:, 2] == dim]
        
        if len(dim_points) > 0:
            births = dim_points[:, 0]
            deaths = dim_points[:, 1]
            
            ax.scatter(births, deaths, c=colors[dim], label=dimensions[dim], 
                      alpha=0.7, s=50)
    
    # Diagonal çizgi
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),
        np.max([ax.get_xlim(), ax.get_ylim()])
    ]
    ax.plot(lims, lims, 'k--', alpha=0.5, zorder=0)
    
    ax.set_xlabel('Doğum (Birth)')
    ax.set_ylabel('Ölüm (Death)')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_topological_features(features_dict, title="Topolojik Özellikler"):
    """
    Topolojik özellikleri görselleştir
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Betti sayıları
    axes[0, 0].bar(range(len(features_dict['betti_numbers'])), 
                   features_dict['betti_numbers'])
    axes[0, 0].set_title('Betti Sayıları')
    axes[0, 0].set_xlabel('Boyut')
    axes[0, 0].set_ylabel('Sayı')
    axes[0, 0].set_xticks(range(len(features_dict['betti_numbers'])))
    
    # Entropy
    entropy = features_dict['entropy']
    axes[0, 1].bar(range(len(entropy)), entropy)
    axes[0, 1].set_title('Kalıcılık Entropisi')
    axes[0, 1].set_xlabel('Boyut')
    axes[0, 1].set_ylabel('Entropi')
    
    # Amplitude
    amplitude = features_dict['amplitude']
    axes[1, 0].bar(range(len(amplitude)), amplitude)
    axes[1, 0].set_title('Kalıcılık Genliği')
    axes[1, 0].set_xlabel('Boyut')
    axes[1, 0].set_ylabel('Genlik')
    
    # Persistence landscape
    landscape = features_dict['landscape']
    axes[1, 1].plot(landscape)
    axes[1, 1].set_title('Kalıcılık Landscape')
    axes[1, 1].set_xlabel('İndeks')
    axes[1, 1].set_ylabel('Değer')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

def compare_topological_features(normal_features, lesion_features):
    """
    Normal ve lezyonlu bölgelerin topolojik özelliklerini karşılaştır
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Betti sayıları karşılaştırması
    x = np.arange(len(normal_features['betti_numbers']))
    width = 0.35
    
    axes[0, 0].bar(x - width/2, normal_features['betti_numbers'], width, 
                   label='Normal', alpha=0.7)
    axes[0, 0].bar(x + width/2, lesion_features['betti_numbers'], width,
                   label='Lezyon', alpha=0.7)
    axes[0, 0].set_title('Betti Sayıları Karşılaştırması')
    axes[0, 0].set_xlabel('Boyut')
    axes[0, 0].set_ylabel('Sayı')
    axes[0, 0].legend()
    
    # Entropy karşılaştırması
    x = np.arange(len(normal_features['entropy']))
    axes[0, 1].bar(x - width/2, normal_features['entropy'], width,
                   label='Normal', alpha=0.7)
    axes[0, 1].bar(x + width/2, lesion_features['entropy'], width,
                   label='Lezyon', alpha=0.7)
    axes[0, 1].set_title('Entropi Karşılaştırması')
    axes[0, 1].set_xlabel('Boyut')
    axes[0, 1].set_ylabel('Entropi')
    axes[0, 1].legend()
    
    # Landscape karşılaştırması
    axes[1, 0].plot(normal_features['landscape'], label='Normal', alpha=0.7)
    axes[1, 0].plot(lesion_features['landscape'], label='Lezyon', alpha=0.7)
    axes[1, 0].set_title('Landscape Karşılaştırması')
    axes[1, 0].set_xlabel('İndeks')
    axes[1, 0].set_ylabel('Değer')
    axes[1, 0].legend()
    
    # Amplitude karşılaştırması
    x = np.arange(len(normal_features['amplitude']))
    axes[1, 1].bar(x - width/2, normal_features['amplitude'], width,
                   label='Normal', alpha=0.7)
    axes[1, 1].bar(x + width/2, lesion_features['amplitude'], width,
                   label='Lezyon', alpha=0.7)
    axes[1, 1].set_title('Genlik Karşılaştırması')
    axes[1, 1].set_xlabel('Boyut')
    axes[1, 1].set_ylabel('Genlik')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

## Örnek Uygulama

In [ ]:
# Topolojik özellik çıkarıcıyı başlat
extractor = TopologicalFeatureExtractor(method='cubical', max_dimension=2)

# Test için örnek görüntüler oluştur
def create_test_images():
    """
    Test için örnek CT görüntüleri oluştur
    """
    # Normal beyin dokusu (düzgün doku)
    normal_img = np.random.normal(0.3, 0.1, (128, 128))
    normal_img = np.clip(normal_img, 0, 1)
    
    # Gaussian blur uygula (daha düzgün doku)
    normal_img = cv2.GaussianBlur(normal_img, (5, 5), 1.0)
    
    # Lezyonlu bölge (karmaşık yapı)
    lesion_img = normal_img.copy()
    
    # Merkeze karmaşık lezyon ekle
    center = (64, 64)
    
    # Ana lezyon (düşük yoğunluk)
    cv2.circle(lesion_img, center, 25, 0.1, -1)
    
    # İç yapılar (yüksek yoğunluk noktalar)
    for i in range(5):
        x = center[0] + np.random.randint(-15, 15)
        y = center[1] + np.random.randint(-15, 15)
        cv2.circle(lesion_img, (x, y), np.random.randint(2, 5), 0.8, -1)
    
    # Kenar efektleri
    cv2.circle(lesion_img, center, 25, 0.6, 2)
    
    return normal_img, lesion_img

# Test görüntülerini oluştur
normal_img, lesion_img = create_test_images()

# Görüntüleri göster
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(normal_img, cmap='gray')
axes[0].set_title('Normal Beyin Dokusu')
axes[0].axis('off')

axes[1].imshow(lesion_img, cmap='gray')
axes[1].set_title('Lezyonlu Beyin Dokusu')
axes[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Normal dokuden topolojik özellikler çıkar
print("Normal doku topolojik analizi...")
normal_features, normal_diagram = extractor.extract_topological_features(normal_img)

# Lezyonlu dokuden topolojik özellikler çıkar
print("Lezyonlu doku topolojik analizi...")
lesion_features, lesion_diagram = extractor.extract_topological_features(lesion_img)

print("Topolojik analiz tamamlandı.")

In [ ]:
# Kalıcılık diagramlarını görselleştir
visualize_persistence_diagram(normal_diagram, "Normal Doku - Kalıcılık Diagramı")
visualize_persistence_diagram(lesion_diagram, "Lezyonlu Doku - Kalıcılık Diagramı")

In [ ]:
# Topolojik özellikleri görselleştir
plot_topological_features(normal_features, "Normal Doku - Topolojik Özellikler")
plot_topological_features(lesion_features, "Lezyonlu Doku - Topolojik Özellikler")

In [ ]:
# Karşılaştırmalı analiz
compare_topological_features(normal_features, lesion_features)

## Topolojik Özellik Vektörleri

In [ ]:
def flatten_topological_features(features):
    """
    Topolojik özellikleri düz vektöre çevir (sinir ağı girişi için)
    """
    feature_vector = []
    
    # Betti sayıları
    feature_vector.extend(features['betti_numbers'].flatten())
    
    # Entropy
    feature_vector.extend(features['entropy'].flatten())
    
    # Amplitude
    feature_vector.extend(features['amplitude'].flatten())
    
    # Number of points
    feature_vector.extend(features['num_points'].flatten())
    
    # Landscape (ilk 10 değer)
    landscape_summary = features['landscape'][:10] if len(features['landscape']) >= 10 else features['landscape']
    if len(landscape_summary) < 10:
        landscape_summary = np.pad(landscape_summary, (0, 10 - len(landscape_summary)))
    feature_vector.extend(landscape_summary)
    
    return np.array(feature_vector)

# Özellik vektörlerini hesapla
normal_vector = flatten_topological_features(normal_features)
lesion_vector = flatten_topological_features(lesion_features)

print(f"Normal doku özellik vektörü boyutu: {len(normal_vector)}")
print(f"Lezyonlu doku özellik vektörü boyutu: {len(lesion_vector)}")

# Özellik vektörlerini karşılaştır
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
x = np.arange(len(normal_vector))
width = 0.35

ax.bar(x - width/2, normal_vector, width, label='Normal', alpha=0.7)
ax.bar(x + width/2, lesion_vector, width, label='Lezyon', alpha=0.7)

ax.set_xlabel('Özellik İndeksi')
ax.set_ylabel('Değer')
ax.set_title('Topolojik Özellik Vektörleri Karşılaştırması')
ax.legend()

plt.tight_layout()
plt.show()

# Euclidean mesafe
distance = np.linalg.norm(normal_vector - lesion_vector)
print(f"Özellik vektörleri arası Euclidean mesafe: {distance:.4f}")

## Topological Feature Layer (PyTorch Modülü)

In [ ]:
class TopologicalLayer(nn.Module):
    """
    Topolojik özellik çıkarımı için PyTorch layer
    """
    
    def __init__(self, method='cubical', max_dimension=2, feature_dim=32):
        super(TopologicalLayer, self).__init__()
        
        self.extractor = TopologicalFeatureExtractor(method, max_dimension)
        self.feature_dim = feature_dim
        
        # Topolojik özellikleri işlemek için MLP
        # Giriş boyutu: yaklaşık 19 özellik (örnek için)
        self.mlp = nn.Sequential(
            nn.Linear(19, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, feature_dim),
            nn.ReLU()
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): Batch of images (B, C, H, W)
            
        Returns:
            torch.Tensor: Topological features (B, feature_dim)
        """
        batch_size = x.shape[0]
        topological_features = []
        
        for i in range(batch_size):
            # Tek görüntü al
            img = x[i, 0].cpu().numpy()  # İlk kanal (grayscale)
            
            # Topolojik özellikler çıkar
            features, _ = self.extractor.extract_topological_features(img)
            
            # Düz vektöre çevir
            feature_vector = flatten_topological_features(features)
            
            # Eksik boyutları doldur veya fazla boyutları kırp
            if len(feature_vector) < 19:
                feature_vector = np.pad(feature_vector, (0, 19 - len(feature_vector)))
            elif len(feature_vector) > 19:
                feature_vector = feature_vector[:19]
            
            topological_features.append(feature_vector)
        
        # Tensor'e çevir
        topological_features = torch.FloatTensor(topological_features).to(x.device)
        
        # MLP ile işle
        processed_features = self.mlp(topological_features)
        
        return processed_features

# Test topological layer
print("Topological Layer testi...")
topo_layer = TopologicalLayer(feature_dim=32)

# Test tensor oluştur
test_batch = torch.FloatTensor(2, 1, 64, 64)  # 2 görüntülük batch
test_batch[0, 0] = torch.from_numpy(cv2.resize(normal_img, (64, 64)))
test_batch[1, 0] = torch.from_numpy(cv2.resize(lesion_img, (64, 64)))

# Forward pass
with torch.no_grad():
    topo_features = topo_layer(test_batch)
    print(f"Topological features shape: {topo_features.shape}")
    print(f"Normal vs Lesion feature distance: {torch.norm(topo_features[0] - topo_features[1]).item():.4f}")

## Sonraki Adımlar

Bu notebook'ta topolojik özellik çıkarımı için gerekli araçları geliştirdik:

1. **Kalıcı homoloji hesaplama**
2. **Topolojik özellik vektörleri**
3. **PyTorch uyumlu topolojik layer**

Sonraki notebook'ta bu topolojik özellikleri U-Net mimarisiyle birleştireceğiz.